In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import os

# ========================================
# 경로 설정 (여기를 수정하세요)
# ========================================
DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/2025ML+DL Programming/dataset/cifar10_data'  # CIFAR-10 데이터를 저장할 경로

# 데이터 디렉토리 생성
os.makedirs(DATA_DIR, exist_ok=True)

# --- CIFAR-10 로드 (자동으로 캐싱) ---
# Keras는 기본적으로 ~/.keras/datasets에 저장하지만,
# 명시적으로 경로를 지정하려면 아래와 같이 처리
cifar10_path = os.path.join(DATA_DIR,'cifar-10-batches-py')

if os.path.exists(cifar10_path):
    print(f"✓ 기존 데이터 발견: {cifar10_path}")
    print("저장된 데이터를 로드합니다...")
else:
    print(f"데이터가 없습니다. {DATA_DIR}에 다운로드합니다...")

# 데이터 로드 (없으면 자동 다운로드 후 캐싱)
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()

print(f"✓ 데이터 로드 완료")
print(f"  - 학습 데이터: {x_train.shape}")
print(f"  - 테스트 데이터: {x_test.shape}\n")

데이터가 없습니다. /content/drive/MyDrive/Colab Notebooks/2025ML+DL Programming/dataset/cifar10_data에 다운로드합니다...
✓ 데이터 로드 완료
  - 학습 데이터: (50000, 32, 32, 3)
  - 테스트 데이터: (10000, 32, 32, 3)



In [13]:
# 클래스 이름 정의
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

# 사용할 클래스 선택
selected_classes = ['cat','dog','horse']
selected_idx = [class_names.index(c)for c in selected_classes]

# --- 해당 클래스만 필터링 ---
train_mask = np.isin(y_train, selected_idx).flatten()
test_mask = np.isin(y_test, selected_idx).flatten()

x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]

print(f"선택된 클래스: {selected_classes}")
print(f"  - 학습 샘플 수: {len(x_train)}")
print(f"  - 테스트 샘플 수: {len(x_test)}\n")

# 라벨을 0~2로 다시 매핑
label_map = {v: i for i, v in enumerate(selected_idx)}
y_train = np.array([label_map[int(y)]for y in y_train])
y_test = np.array([label_map[int(y)]for y in y_test])

# 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0

선택된 클래스: ['cat', 'dog', 'horse']
  - 학습 샘플 수: 15000
  - 테스트 샘플 수: 3000



/tmp/ipython-input-101200204.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_train = np.array([label_map[int(y)]for y in y_train])
/tmp/ipython-input-101200204.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_test = np.array([label_map[int(y)]for y in y_test])


In [14]:
x_train.shape

(15000, 32, 32, 3)

In [19]:
from tensorflow.keras import Sequential
model = models.Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),  # ★ 3채널
    MaxPooling2D(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [21]:
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1,
    verbose=2
)

Epoch 1/10
106/106 - 17s - 159ms/step - accuracy: 0.5092 - loss: 0.9719 - val_accuracy: 0.5773 - val_loss: 0.8748
Epoch 2/10
106/106 - 20s - 187ms/step - accuracy: 0.6233 - loss: 0.7976 - val_accuracy: 0.6593 - val_loss: 0.7495
Epoch 3/10
106/106 - 15s - 142ms/step - accuracy: 0.6759 - loss: 0.7249 - val_accuracy: 0.6573 - val_loss: 0.7508
Epoch 4/10
106/106 - 15s - 141ms/step - accuracy: 0.6953 - loss: 0.6785 - val_accuracy: 0.6940 - val_loss: 0.6835
Epoch 5/10
106/106 - 15s - 141ms/step - accuracy: 0.7182 - loss: 0.6421 - val_accuracy: 0.7153 - val_loss: 0.6459
Epoch 6/10
106/106 - 15s - 139ms/step - accuracy: 0.7299 - loss: 0.6164 - val_accuracy: 0.7013 - val_loss: 0.6562
Epoch 7/10
106/106 - 21s - 195ms/step - accuracy: 0.7473 - loss: 0.5862 - val_accuracy: 0.7100 - val_loss: 0.6392
Epoch 8/10
106/106 - 15s - 138ms/step - accuracy: 0.7551 - loss: 0.5649 - val_accuracy: 0.7340 - val_loss: 0.6295
Epoch 9/10
106/106 - 21s - 194ms/step - accuracy: 0.7687 - loss: 0.5381 - val_accuracy: 

In [22]:
# 7️⃣ 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

94/94 - 2s - 23ms/step - accuracy: 0.7463 - loss: 0.5884

✅ Test Accuracy: 0.7463


In [24]:
# 8️⃣ 예측 예시
predictions = model.predict(x_test[:20])
print("예측 결과:", np.argmax(predictions, axis=1))
print("실제 레이블:", y_test[:20])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
예측 결과: [0 0 0 2 1 2 2 1 1 0 1 2 1 2 0 1 1 2 1 0]
실제 레이블: [0 0 1 2 1 2 2 1 1 1 1 1 0 2 0 2 2 2 0 0]
